# **Waste Material Segregation for Improving Waste Management**

## **Objective**

The objective of this project is to implement an effective waste material segregation system using convolutional neural networks (CNNs) that categorises waste into distinct groups. This process enhances recycling efficiency, minimises environmental pollution, and promotes sustainable waste management practices.

The key goals are:

* Accurately classify waste materials into categories like cardboard, glass, paper, and plastic.
* Improve waste segregation efficiency to support recycling and reduce landfill waste.
* Understand the properties of different waste materials to optimise sorting methods for sustainability.

## **Data Understanding**

The Dataset consists of images of some common waste materials.

1. Food Waste
2. Metal
3. Paper
4. Plastic
5. Other
6. Cardboard
7. Glass


**Data Description**

* The dataset consists of multiple folders, each representing a specific class, such as `Cardboard`, `Food_Waste`, and `Metal`.
* Within each folder, there are images of objects that belong to that category.
* However, these items are not further subcategorised. <br> For instance, the `Food_Waste` folder may contain images of items like coffee grounds, teabags, and fruit peels, without explicitly stating that they are actually coffee grounds or teabags.

## **1. Load the data**

Load and unzip the dataset zip file.

**Import Necessary Libraries**

In [ ]:
# Recommended versions:

# numpy version: 1.26.4
# pandas version: 2.2.2
# seaborn version: 0.13.2
# matplotlib version: 3.10.0
# PIL version: 11.1.0
# tensorflow version: 2.18.0
# keras version: 3.8.0
# sklearn version: 1.6.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import essential libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split


Load the dataset.

In [ ]:
import zipfile

# Use raw strings to avoid escape character issues
zip_path = '/content/drive/MyDrive/CNN_assignment/data.zip'
extract_path = '/content/dataset'

# Check and extract the zip file
if os.path.exists(zip_path) and zipfile.is_zipfile(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f" Dataset extracted to: {extract_path}")
else:
    print(" Zip file not found or invalid format.")


## **2. Data Preparation** <font color=red> [25 marks] </font><br>


### **2.1 Load and Preprocess Images** <font color=red> [8 marks] </font><br>

Let us create a function to load the images first. We can then directly use this function while loading images of the different categories to load and crop them in a single step.

#### **2.1.1** <font color=red> [3 marks] </font><br>
Create a function to load the images.

In [ ]:
def load_images_from_directory(base_dir, image_size=(128, 128)):
    images = []
    labels = []

    for class_name in os.listdir(base_dir + "/data"):  # Subfolders are class names
        class_path = os.path.join(base_dir, "data", class_name)
        if os.path.isdir(class_path):  # Ensure it's a directory
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                try:
                    img = Image.open(img_path).convert('RGB')  # Make sure image is RGB
                    img = img.resize(image_size)
                    images.append(np.array(img))
                    labels.append(class_name)
                except Exception as e:
                    print(f"❌ Could not process file {img_path}: {e}")

    return images, labels


#### **2.1.2** <font color=red> [5 marks] </font><br>
Load images and labels.

In [ ]:
# Load images and labels

dataset_path = '/content/dataset'
image_size = (128, 128)

images, labels = load_images_from_directory(dataset_path, image_size)
images = np.array(images)
labels = np.array(labels)

print(f"Total images loaded: {len(images)}")
print(f"Unique labels found: {set(labels)}")


Load the images from the dataset directory. Labels of images are present in the subdirectories.

Verify if the images and labels are loaded correctly.

In [ ]:
# Convert labels to numpy array (if not already done)
labels = np.array(labels)

# Verify if data looks correct
print(f"Image data shape: {images.shape}")
print(f"First 5 labels: {labels[:5]}")

# Show 5 sample images with labels
plt.figure(figsize=(10, 5))
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.imshow(images[i].astype('uint8'))  # Ensures correct display
    plt.title(labels[i])
    plt.axis('off')
plt.tight_layout()
plt.show()


Perform any operations, if needed, on the images and labels to get them into the desired format.

### **2.2 Data Visualisation** <font color=red> [9 marks] </font><br>

#### **2.2.1** <font color=red> [3 marks] </font><br>
Create a bar plot to display the class distribution

In [ ]:
# Visualise Data Distribution

# Convert labels list to a pandas Series for easier plotting
label_series = pd.Series(labels)

# Plot class distribution
plt.figure(figsize=(10, 6))
sns.countplot(x=label_series)
plt.title("Class Distribution")
plt.xlabel("Class")
plt.ylabel("Number of Images")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



#### **2.2.2** <font color=red> [3 marks] </font><br>
Visualise some sample images

In [ ]:
# Visualise Sample Images (across different labels)
import random

# Get unique classes
classes = sorted(set(labels))

# Plot sample images from each class
plt.figure(figsize=(15, 10))
for i, class_name in enumerate(classes):
    # Find indices where label == class_name
    class_indices = [idx for idx, lbl in enumerate(labels) if lbl == class_name]
    img_idx = random.choice(class_indices)

    plt.subplot(2, 4, i+1)  # adjust rows/cols depending on number of classes
    plt.imshow(images[img_idx])
    plt.title(class_name)
    plt.axis('off')

plt.suptitle("Sample Images from Each Class")
plt.tight_layout()
plt.show()



Check for Image size consistency

In [ ]:
# Check that all images have the same shape
unique_shapes = set([img.shape for img in images])
print(f"Unique image shapes in dataset: {unique_shapes}")


Flatten all image pixels to view distribution


#### **2.2.3** <font color=red> [3 marks] </font><br>
Based on the smallest and largest image dimensions, resize the images.

In [ ]:
# Find the smallest and largest image dimensions from the data set
# Gather width and height of all images
widths = []
heights = []

for img in images:
    h, w = img.shape[:2]
    widths.append(w)
    heights.append(h)

# Find min and max
min_w, max_w = min(widths), max(widths)
min_h, max_h = min(heights), max(heights)

print(f"Smallest image dimensions: ({min_h}, {min_w})")
print(f"Largest image dimensions: ({max_h}, {max_w})")



In [ ]:
# Resize the image dimensions
# Resize all images to a consistent shape (128, 128)
resized_images = []

for img in images:
    img_resized = np.array(Image.fromarray((img * 255).astype(np.uint8)).resize(image_size))
    # Normalize the resized image back to [0, 1]
    img_resized = img_resized / 255.0
    resized_images.append(img_resized)

# Convert to numpy array
resized_images = np.array(resized_images)

print(f"All images resized to: {resized_images[0].shape}")



### **2.3 Encoding the classes** <font color=red> [3 marks] </font><br>

There are seven classes present in the data.



We have extracted the images and their labels, and visualised their distribution. Now, we need to perform encoding on the labels. Encode the labels suitably.

####**2.3.1** <font color=red> [3 marks] </font><br>
Encode the target class labels.

In [ ]:
# Encode the labels suitably
# Get unique classes
unique_classes = sorted(set(labels))
label_to_index = {label: idx for idx, label in enumerate(unique_classes)}

# Integer encode the labels
encoded_labels = np.array([label_to_index[label] for label in labels])

print("Label to index mapping:", label_to_index)
print("First 5 encoded labels:", encoded_labels[:5])



In [ ]:
# Integer encoding for labels
unique_classes = sorted(set(labels))
label_to_index = {label: idx for idx, label in enumerate(unique_classes)}
encoded_labels = np.array([label_to_index[label] for label in labels])

# Plot encoded label distribution
plt.figure(figsize=(8, 5))
sns.countplot(x=encoded_labels)
plt.title("Encoded Label Distribution")
plt.xlabel("Encoded Label")
plt.ylabel("Count")
plt.xticks(ticks=range(len(unique_classes)), labels=unique_classes, rotation=45)
plt.tight_layout()
plt.show()


### **2.4 Data Splitting** <font color=red> [5 marks] </font><br>

#### **2.4.1** <font color=red> [5 marks] </font><br>
Split the dataset into training and validation sets

In [ ]:
# Assign specified parts of the dataset to train and validation sets

num_classes = len(unique_classes)
one_hot_labels = np.eye(num_classes)[encoded_labels]

# Split the data
X_train, X_val, y_train, y_val = train_test_split(
    images, one_hot_labels, test_size=0.2, random_state=42, stratify=encoded_labels
)

print("Training set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_val.shape, y_val.shape)

## **3. Model Building and Evaluation** <font color=red> [20 marks] </font><br>

### **3.1 Model building and training** <font color=red> [15 marks] </font><br>

#### **3.1.1** <font color=red> [10 marks] </font><br>
Build and compile the model. Use 3 convolutional layers. Add suitable normalisation, dropout, and fully connected layers to the model.

Test out different configurations and report the results in conclusions.

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, BatchNormalization


# Model 1

In [ ]:
# Build and compile the model
model1 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.2),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.3),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.3),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(7, activation='softmax')
])

model1.summary()


Add Augmentation to improve generalization

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% for validation
)

train_generator = train_datagen.flow(
    X_train, y_train,
    batch_size=32,
    subset='training'
)

val_generator = train_datagen.flow(
    X_train, y_train,
    batch_size=32,
    subset='validation'
)

#### **3.1.2** <font color=red> [5 marks] </font><br>
Train the model.

Use appropriate metrics and callbacks as needed.

In [ ]:
# Compile the model
model1.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Callbacks

In [ ]:
checkpoint1 = ModelCheckpoint(
    "best_model1.keras",
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

early_stop1 = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr1 = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    verbose=1,
    min_lr=1e-5
)

In [ ]:
# Train the model
history1 = model1.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=[checkpoint, early_stop, reduce_lr]
)

In [ ]:
def plot_history(history):
    plt.figure(figsize=(14, 5))

    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Model Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_history(history1)

# Model 2


In [ ]:

model2 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.2),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.3),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.3),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(7, activation='softmax')
])

#Model 2 Summary

model2.summary()

In [ ]:
# Training Model 2

# Compile the model1

model2.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

checkpoint2 = ModelCheckpoint(
    "best_model2.keras",
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

early_stop2 = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr2 = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    verbose=1,
    min_lr=1e-5
)

In [ ]:
# Train the model2
history2 = model2.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=[checkpoint2, early_stop2, reduce_lr2]
)

# Plot History of Model 2
plot_history(history2)


# Model 3

In [ ]:
model3 = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Dropout(0.2),

    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.3),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.4),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(7, activation='softmax')
])


In [ ]:
model3.summary()


In [ ]:
# Training Model 3

# Compile the model 3

model3.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']


checkpoint3 = ModelCheckpoint(
    "best_model3.keras",
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

early_stop3 = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr3 = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    verbose=1,
    min_lr=1e-5
)


In [ ]:

# Train the model3
history3 = model3.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=[checkpoint3, early_stop3, reduce_lr3]
)

# Plot History of Model 3
plot_history(history3)

### **3.2 Model Testing and Evaluation** <font color=red> [5 marks] </font><br>

#### **3.2.1** <font color=red> [5 marks] </font><br>
Evaluate the model on test dataset. Derive appropriate metrics.

In [ ]:
# Evaluate on the test set; display suitable metrics



## **4. Data Augmentation** <font color=red> [optional] </font><br>

#### **4.1 Create a Data Augmentation Pipeline**

##### **4.1.1**
Define augmentation steps for the datasets.

In [ ]:
# Define augmentation steps to augment images



Augment and resample the images.
In case of class imbalance, you can also perform adequate undersampling on the majority class and augment those images to ensure consistency in the input datasets for both classes.

Augment the images.

In [ ]:
# Create a function to augment the images




In [ ]:
# Create the augmented training dataset



##### **4.1.2**

Train the model on the new augmented dataset.

In [ ]:
# Train the model using augmented images



## **5. Conclusions** <font color = red> [5 marks]</font>

#### **5.1 Conclude with outcomes and insights gained** <font color =red> [5 marks] </font>

* Report your findings about the data
* Report model training results